In [7]:
%run -i writefile2.py

In [8]:
%%writefile2
import require
from tqdm import tqdm
import sys
import nodes
from IPython.display import display, HTML
from types import SimpleNamespace as ns

"""
collects training data from a selection of countries as a list of dataframes

Note: by cropping here we loose some data (temporally), 
but we use the dataframe for window processing and need NaN-free series
"""

#TODO: can be replaced by cached nodes
compute_data_for_country = require.single( "compute_data_for_country" )
crop_to_valid_range = require.single( "crop_to_valid_range" )
verbose = require.untracked.single( "verbose" )
owid_countries = require.single( "countries_in_owid" )
oxford_countries = require.single( "countries_in_oxford" )
broken_countries = require.single( "countries_known_to_be_broken" )
learnable_countries = require.single( "countries_with_high_r2" )

def compute_training_data( ):

    countries = learnable_countries
    countries.sort( )
    
    if verbose( ):

        display( HTML( f"<h1>Training data</h1>" ))
        print( countries )
    
    dataframes = [ ]

    for country in tqdm( countries, file = sys.stdout, desc = "collecting data" ):
        
        if verbose( ):
        
            display( HTML( f"<h3>{ country }</h3>" ))

        df = compute_data_for_country( country )
        df = crop_to_valid_range( df ).copy( ) 
        dataframes.append( ns( df = df, country = country ))

    return dataframes
        

In [4]:
%%writefile2 --source training_data_h.ipynb

"""
cache node wrapper around compute_training_data.py
"""

import nodes
import require

@nodes.store_in_tmp
@nodes.generic_node
def training_data( ):

    compute_training_data = require.single( "compute_training_data" )
    
    def main( ):

        return compute_training_data( )

    return main

node = training_data

In [7]:
import verbose as v
v.set_level( 0 )
compute_training_data( )

collecting data: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 123/123 [00:41<00:00,  2.94it/s]


[            new_cases_smoothed_per_million  new_deaths_smoothed_per_million  \
 date                                                                          
 2020-01-08                           0.000                            0.000   
 2020-01-09                           0.000                            0.000   
 2020-01-10                           0.000                            0.000   
 2020-01-11                           0.000                            0.000   
 2020-01-12                           0.000                            0.000   
 ...                                    ...                              ...   
 2023-05-13                           0.359                            0.003   
 2023-05-14                           0.359                            0.003   
 2023-05-15                           0.000                            0.000   
 2023-05-16                           0.000                            0.000   
 2023-05-17                           0.